# Base de datos Cassandra

## Conexion y borrado de datos

In [1]:
%load_ext cql

In [2]:
%%cql
DROP KEYSPACE practica_8anu;

'No results.'

In [3]:
%%cql
CREATE KEYSPACE practica_8anu 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [4]:
%cql USE practica_8anu;

'No results.'

## Creacion de tablas

Para la resolución de las cuestiones he utilizado utilizado tablas, tablas de acumulación y vistas materializadas con el objetivo resolver las preuntas utilizando al máximo posible la funcionalidad que la propia base de datos ofrece.

En las imagenes siguientes aparecen por orden las claves primarias, siendo las rojas claves de partición y las azules claves de clustering. Si además se ha solicitado un orden en el clustering, se especifica en qué columna(s) se ha(n) pedido y con que ordenacion(ASC o DESC).

Como se va a poder observar, las tablas aparecen desnormalizadas, repitiendo información según la información que se ha querido visualizar en las respuestas a las preguntas.

Las **tablas "normales"** creadas han sido estas:

![Tablas](./img/cassandra1.JPG)

**TABLAS DE ACUMULACIÓN**

Las **tablas de acumulacion** las he realizado debido a que algunas preguntas requerían la aplicación de funciones de agregacion (como la media) y agrupación por los valores de alguna columna. Cassandra ofrece la posibilidad de aplicar funciones de agregación (como se ve en alguna respuesta) pero no de agrupación. Para resolver esto he recurrido a las tablas de acumulación y así probar esta funcionalidad que ofrece Cassandra. Es remarcable que este tipo de situacion agregacion+agrupación tambien es resoluble realizando tablas en las que solo existieran elementos de un tipo en función de la columna sobre la que se solicta la agrupación.

![Tablas acumulacion](./img/cassandra2.JPG)

**VISTAS MATERIALIZADAS**

He recurrido intensivamente al **uso de vistas materializadas sobre la tabla de ascensos** que es la que más informacion tiene. Lo he hecho asi con el fin de explotar al maximo posible la funcionalidad que esta base de datos ofrece. Esto se debe a que sobre los ascensos se realizan muchas preguntas de muy diferente indole, en las que a veces interesa un acceso y/o una ordenación diferente.

![Vistas materializadas](./img/cassandra3.JPG)
![Vistas materializadas](./img/cassandra4.JPG)
![Vistas materializadas](./img/cassandra5.JPG)



** TABLA ESCALADORES, ASCENSOS Y DIFICULTADES **

In [5]:
%%cql 
CREATE TABLE escaladores (
    id_escalador    int,
    nombre          text,
    sexo            text,
    pais            text,
    anio_comienzo   int,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [6]:
%%cql 
CREATE TABLE ascensos (
    uid                  uuid,
    id_escalador         int,
    nombre_escalador     text,
    pais_escalador       text,
    anio_comienzo        int,
    nombre_via           text,
    dificultad           int,
    grado_frances        text,
    tipo_encadenamiento  text,
    risco                text,
    pais_risco           text,
    PRIMARY KEY (id_escalador, tipo_encadenamiento, dificultad, pais_risco, uid)
);

'No results.'

In [7]:
%%cql 
CREATE TABLE dificultades (
    dificultad    int,
    grado_frances text,
    PRIMARY KEY (dificultad)
);

'No results.'

** TABLA DE ACUMULACION **

**Ascensos acumulados por sexo y escalador**

In [8]:
%%cql 
CREATE TABLE ascensos_acumulados_hombres (
    id_escalador    int,
    num_ascensos    counter,
    dificultad_acum counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [9]:
%%cql 
CREATE TABLE ascensos_acumulados_mujeres (
    id_escalador    int,
    num_ascensos    counter,
    dificultad_acum counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

** Tablas de ascensos acumulados de escaladores españoles y no españoles en España: **

In [10]:
%%cql 
CREATE TABLE ascensos_acum_esp_esp (
    id_escalador    int,
    nombre_escalador text,
    pais_escalador   text,
    num_ascensos    counter,
    PRIMARY KEY (id_escalador, nombre_escalador, pais_escalador)
);

'No results.'

In [11]:
%%cql 
CREATE TABLE ascensos_acum_noesp_esp (
    id_escalador    int,
    nombre_escalador text,
    pais_escalador   text,
    num_ascensos    counter,
    PRIMARY KEY (id_escalador, nombre_escalador, pais_escalador )
);

'No results.'

**Ascensos acumulados por risco y pais:**

In [12]:
%%cql 
CREATE TABLE ascensos_acum_pais_risco (
    pais_risco      text,
    risco           text,
    num_ascensos    counter,
    dificultad_acum counter,
    PRIMARY KEY (pais_risco, risco)
);

'No results.'

** VISTAS MATERIALIZADAS **

In [13]:
%%cql 
CREATE MATERIALIZED VIEW ascensos_por_encadenamiento_escalador AS
    SELECT * FROM ascensos
    WHERE uid IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND id_escalador IS NOT NULL
        AND nombre_escalador IS NOT NULL
        AND pais_escalador IS NOT NULL
        AND anio_comienzo IS NOT NULL
        AND nombre_via IS NOT NULL
        AND dificultad IS NOT NULL
        AND grado_frances IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND risco IS NOT NULL
        AND pais_risco IS NOT NULL 
    PRIMARY KEY (tipo_encadenamiento, id_escalador, dificultad, nombre_via, pais_risco, uid) 
    WITH CLUSTERING ORDER BY (dificultad DESC, nombre_via ASC);

'No results.'

In [14]:
%%cql 
CREATE MATERIALIZED VIEW ascensos_por_encadenamiento_dificultad AS
    SELECT * FROM ascensos
    WHERE uid IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND id_escalador IS NOT NULL
        AND nombre_escalador IS NOT NULL
        AND pais_escalador IS NOT NULL
        AND anio_comienzo IS NOT NULL
        AND nombre_via IS NOT NULL
        AND dificultad IS NOT NULL
        AND grado_frances IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND risco IS NOT NULL
        AND pais_risco IS NOT NULL 
    PRIMARY KEY (tipo_encadenamiento, dificultad, id_escalador, nombre_via, pais_risco, uid) 
    WITH CLUSTERING ORDER BY (dificultad DESC);

'No results.'

In [15]:
%%cql 
CREATE MATERIALIZED VIEW ascensos_por_escalador_paisRisco AS
    SELECT * FROM ascensos
    WHERE uid IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND id_escalador IS NOT NULL
        AND nombre_escalador IS NOT NULL
        AND pais_escalador IS NOT NULL
        AND anio_comienzo IS NOT NULL
        AND nombre_via IS NOT NULL
        AND dificultad IS NOT NULL
        AND grado_frances IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND risco IS NOT NULL
        AND pais_risco IS NOT NULL 
    PRIMARY KEY (id_escalador, pais_risco, dificultad, tipo_encadenamiento, nombre_via, uid) 
    WITH CLUSTERING ORDER BY (dificultad DESC);

'No results.'

In [16]:
%%cql 
CREATE MATERIALIZED VIEW ascensos_por_encadenamiento_aniocomienzo AS
    SELECT * FROM ascensos
    WHERE uid IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND id_escalador IS NOT NULL
        AND nombre_escalador IS NOT NULL
        AND pais_escalador IS NOT NULL
        AND anio_comienzo IS NOT NULL
        AND nombre_via IS NOT NULL
        AND dificultad IS NOT NULL
        AND grado_frances IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND risco IS NOT NULL
        AND pais_risco IS NOT NULL 
    PRIMARY KEY (tipo_encadenamiento, anio_comienzo, id_escalador, pais_risco, dificultad, uid);

'No results.'

In [17]:
%%cql 
CREATE MATERIALIZED VIEW ascensos_por_pais AS
    SELECT * FROM ascensos
    WHERE uid IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND id_escalador IS NOT NULL
        AND nombre_escalador IS NOT NULL
        AND pais_escalador IS NOT NULL
        AND anio_comienzo IS NOT NULL
        AND nombre_via IS NOT NULL
        AND dificultad IS NOT NULL
        AND grado_frances IS NOT NULL
        AND tipo_encadenamiento IS NOT NULL
        AND risco IS NOT NULL
        AND pais_risco IS NOT NULL 
    PRIMARY KEY (pais_risco, tipo_encadenamiento, id_escalador, dificultad, uid);

'No results.'

## Lectura de la informacion en Pandas

In [18]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
columnas = ['id_escalador',
            'id_dificultad',
            'tipo_encadenamiento',
            'nombre_via',
            'risco',
            'sector',
            'pais']
df_ascensos = df_ascensos[columnas]

## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver

In [19]:
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'left');
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'left');

# Renombrar campos por claridad
df_ascensos = df_ascensos.rename(
    columns={
        'id_dificultad' : 'dificultad',
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});


## Carga de datos en Cassandra

In [20]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('practica_8anu')

**CARGA EN TABLAS DE ASCENSOS Y ACUMULADOS**

In [21]:
def insert_ascensos(df):
    cql_insert_ascenso = """
        INSERT INTO ascensos (
            uid,
            id_escalador,
            nombre_escalador,
            pais_escalador,
            anio_comienzo,
            nombre_via,
            dificultad,
            grado_frances,
            tipo_encadenamiento,
            risco,
            pais_risco
        ) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    

    
    for index in df.index:
        data = [
            df.ix[index,"id_escalador"],
            df.ix[index,"nombre_escalador"],
            df.ix[index,"pais_escalador"],
            df.ix[index,"anio_comienzo"],
            df.ix[index,"nombre_via"],
            df.ix[index,"dificultad"],
            df.ix[index,"grado_frances"],
            df.ix[index,"tipo_encadenamiento"],
            df.ix[index,"risco"],
            df.ix[index,"pais_risco"]
        ]

        session.execute(cql_insert_ascenso, data)
        
        session.execute("""UPDATE ascensos_acum_pais_risco
                            SET num_ascensos = num_ascensos + 1,
                                dificultad_acum = dificultad_acum + %s
                            WHERE pais_risco = %s AND risco = %s""",
                        [df.ix[index,"dificultad"],
                         df.ix[index,"pais_risco"],
                         df.ix[index,"risco"]])
        
        
        if (df.ix[index,"sexo"] == 'Hombre'):
            session.execute("""UPDATE ascensos_acumulados_hombres
                               SET num_ascensos = num_ascensos + 1,
                                   dificultad_acum = dificultad_acum + %s
                               WHERE id_escalador = %s""",
                            [df.ix[index,"dificultad"], df.ix[index,"id_escalador"]])
        else:
            session.execute("""UPDATE ascensos_acumulados_mujeres
                               SET num_ascensos = num_ascensos + 1,
                                   dificultad_acum = dificultad_acum + %s
                               WHERE id_escalador = %s""",
                            [df.ix[index,"dificultad"], df.ix[index,"id_escalador"]])
            
        if (df.ix[index,"pais_risco"] == 'ESP'):
            if(df.ix[index,"pais_escalador"] == 'ESP'):
                session.execute("""UPDATE ascensos_acum_esp_esp
                                SET num_ascensos = num_ascensos + 1
                                WHERE id_escalador = %s 
                                    AND nombre_escalador = %s
                                    AND pais_escalador = %s""",
                                [df.ix[index,"id_escalador"],
                                 df.ix[index,"nombre_escalador"],
                                 df.ix[index,"pais_escalador"]])
            else:
                session.execute("""UPDATE ascensos_acum_noesp_esp
                                SET num_ascensos = num_ascensos + 1
                                WHERE id_escalador = %s 
                                    AND nombre_escalador = %s
                                    AND pais_escalador = %s""",
                                [df.ix[index,"id_escalador"],
                                 df.ix[index,"nombre_escalador"],
                                 df.ix[index,"pais_escalador"]])
        

In [22]:
insert_ascensos(df_ascensos)

** CARGA EN TABLA DE DIFICULTADES**

In [23]:
def insert_dificultades(df):
    cql_insert = """
        INSERT INTO dificultades (
            dificultad,
            grado_frances
        ) VALUES (%s, %s)"""
    
    for index in df.index:
        data = [df.ix[index, "id_dificultad"],
                df.ix[index, "grado_frances"]]
        session.execute(cql_insert, data)

In [24]:
insert_dificultades(df_dificultades)

** CARGA EN TABLA DE ESCALADORES **

In [25]:
def insert_escaladores(df):
    cql_insert = """
        INSERT INTO escaladores (
            id_escalador,
            nombre,
            sexo,
            pais,
            anio_comienzo
        ) VALUES (%s, %s, %s, %s, %s)"""
    
    for index in df.index:
        data = [
            df.ix[index, "id_escalador"],
            df.ix[index, "nombre"],
            df.ix[index, "sexo"],
            df.ix[index, "pais"],
            df.ix[index, "comienzo"]]
        session.execute(cql_insert, data)

In [26]:
insert_escaladores(df_escaladores)

# RESPUESTAS A LAS PREGUNTAS

In [27]:
def execute_query(cql, data=[]):
    rows = session.execute(cql, data)
    return pd.DataFrame(list(rows))

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

Esto requiere varias consultas a dos tablas diferentes:
- Una consulta a la tabla de acumulados de hombres de la que se obtienen los ids de los escaladores con mas ascensos acumulados
- 10 consultas a la tabla de escaladores, para sacar los datos del escalador.

Por desgracia las tablas de acumulación (con columnas counter) tienen las siguientes particularidades:

- exceptuando una columna que hará de partition key, estas tablas no pueden contener columnas de otro tipo diferente a counter a no ser que esas nuevas columnas se introduzcan como parte de la primary key: https://stackoverflow.com/questions/32562500/cassandra-non-counter-family


- una columna counter no puede ser clustering key por lo que no se puede utilizar como columna de ordenación. Esto hace que la ordenación haya hacerla fuera.


In [28]:
cql = """
SELECT *
FROM ascensos_acumulados_hombres
"""
df_id_acum = execute_query(cql)

df_id_acum = df_id_acum.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, False]).head(10)

In [29]:
cql = """
SELECT *
FROM escaladores
WHERE id_escalador=%s
"""

df = pd.DataFrame()
for index in df_id_acum.index:
    df_escalador = execute_query(cql, [df_id_acum.ix[index,'id_escalador']])
    df_escalador["num_ascensos"] = df_id_acum.ix[index, "num_ascensos"]
    
    df = df.append(df_escalador)
    
df = df.reset_index(drop=True)
id_escalador_mas_activo = df.iloc[0]['id_escalador']

columnas = ["id_escalador", "nombre", "sexo", "pais", "num_ascensos"]
df[columnas]  
    

,id_escalador,nombre,sexo,pais,num_ascensos
0,50884,Christopher Leonetti,Hombre,USA,47
1,20384,Clemens Kurth,Hombre,DEU,26
2,20095,Matthias Schuster,Hombre,DEU,26
3,66250,Kuba Kaminski,Hombre,POL,24
4,9171,Laurenz Trawnicek,Hombre,AUT,23
5,46622,Raúl Crespo,Hombre,ESP,19
6,42086,Marcin Opozda,Hombre,POL,19
7,66466,Christian Boehme,Hombre,DEU,18
8,47732,Thomas de Fleurian,Hombre,FRA,18
9,35847,Zack Bum,Hombre,DEU,18


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

In [30]:
cql = """
SELECT *
FROM ascensos_acumulados_mujeres
"""
df_id_acum = execute_query(cql)

df_id_acum = df_id_acum.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, False]).head(10)

In [31]:
cql = """
SELECT *
FROM escaladores
WHERE id_escalador=%s
"""

df = pd.DataFrame()
for index in df_id_acum.index:
    df_escalador = execute_query(cql, [df_id_acum.ix[index,'id_escalador']])
    df_escalador["num_ascensos"] = df_id_acum.ix[index, "num_ascensos"]
    
    df = df.append(df_escalador)
    
df = df.reset_index(drop=True)

id_escaladora_mas_activa = df.iloc[0]['id_escalador']

columnas = ["id_escalador", "nombre", "sexo", "pais", "num_ascensos"]
df[columnas] 



,id_escalador,nombre,sexo,pais,num_ascensos
0,65502,Ksenia Targosz,Mujer,POL,13
1,65069,Mania Mania A.,Mujer,POL,12
2,65707,Ewelina Cienkus,Mujer,POL,11
3,54695,Elfi Hasler,Mujer,AUT,11
4,62354,La Shoune,Mujer,FRA,10
5,58835,Dominika Sołtys,Mujer,POL,10
6,57447,Sue Murphy,Mujer,CAN,10
7,53983,Daniela Bärtschi,Mujer,CHE,10
8,49569,Karina Kosiorek,Mujer,POL,10
9,32043,Reidun M. Romundstad,Mujer,NOR,10


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

De la pregunta anterior nos hemos guardado el id de la escaladora más activa. 

Para la respuesta a esta pregunta he creado una vista materializada de la tabla de ascensos, ascensos_por_encadenamiento, cuya clave de particion es el tipo de encadenamiento y la ordenacion se hace por dificultad y por nombre de via. 

In [32]:
cql = """
SELECT *
FROM ascensos_por_encadenamiento_escalador
WHERE tipo_encadenamiento = 'Onsight' AND id_escalador = %s 
LIMIT 10
"""

df = execute_query(cql, [id_escaladora_mas_activa])
columnas = ['nombre_via', 'grado_frances', 'risco', 'pais_risco']
df[columnas]

,nombre_via,grado_frances,risco,pais_risco
0,LEWIZNA,5c,DOLINA KOBYLANSKA,POL
1,MOJA KOKAINA,5c,DOLINA KOBYLANSKA,POL
2,RUCHY MASOWE,5c,DOLINA BEDKOWSKA,POL
3,FISCHER-LAPINSKI,5b,DOLINA BEDKOWSKA,POL
4,GRZBIET ZUBRA,5b,SOKOLIKI,POL
5,RYSKOWIEC,5b,DOLINA BEDKOWSKA,POL
6,ZAGłADA TRADA,4c,DOLINA KOBYLANSKA,POL
7,KIKUT,4b,DOLINA SZKLARKI,POL


### 3. Dificultad media de los ascensos del escalador más activo

Esto requiere dos consultas:

- A la tabla ascensos_acumulados_hombres para obtener los datos para poder calcular la dificultad media.
- A la tabla de dificultades_grado para obtener el grado frances de la media


In [33]:
# query a tabla de ascensos acumulados
cql = """
SELECT *
FROM ascensos_acumulados_hombres
WHERE id_escalador = %s
"""

df = execute_query(cql, [id_escalador_mas_activo])
dificultad_media = int(round(df["dificultad_acum"]/df["num_ascensos"]))

# query a la tabla de dificultades
cql = """
SELECT grado_frances
FROM dificultades
WHERE dificultad = %s
"""
execute_query(cql, [dificultad_media])


,grado_frances
0,5c+


### 4.a) Los 10 ascensos mas dificiles

La vista materializada tiene ordenados los **ascensos_por_encadenamiento** para cada tipo de encadenamiento. Podríamos resolver esta cuestion de la manera siguiente:

- Obtener mediante 4 queries los 10 primeros ascensos para cada tipo de encadenamiento
- Unir (append) los data frames resultado
- Reordenar el data frame

Pero en este caso vamos a recurrir a un conocimiento a priori del domino de la escalada: los grados más difíciles se encadenan siempre en "Red point"; esto se debe a que los ascensos más dificiles requieren siempre del escalador un entrenamiento en la propia via, por lo que el ascenso final de la via es "Red point".

In [34]:
cql = """
SELECT *
FROM ascensos_por_encadenamiento_dificultad
WHERE tipo_encadenamiento = 'Redpoint'
LIMIT 10
"""

df = execute_query(cql)
columnas = ['id_escalador','nombre_escalador','pais_escalador','nombre_via','grado_frances','tipo_encadenamiento','risco','pais_risco']
df[columnas]

,id_escalador,nombre_escalador,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
0,1476,Adam Ondra,CZE,SILENCE,9c,Redpoint,FLATANGER,NOR
1,1476,Adam Ondra,CZE,MOVE HARD,9b,Redpoint,FLATANGER,NOR
2,22437,Stefano Ghisolfi,ITA,FIRST ROUND FIRST MINUTE,9b,Redpoint,MARGALEF,ESP
3,1476,Adam Ondra,CZE,NATURALMENTE,9a+,Redpoint,CAMAIORE,ITA
4,1476,Adam Ondra,CZE,ULTIMATUM,9a+,Redpoint,ARCO,ITA
5,8707,Daniel Fuertes,ESP,NO PAIN NO GAIN,9a+,Redpoint,RODELLAR,ESP
6,14130,David Firnenburg,DEU,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
7,18008,Piotr Schab,POL,THOR'S HAMMER,9a+,Redpoint,FLATANGER,NOR
8,22437,Stefano Ghisolfi,ITA,FIRST LEY,9a+,Redpoint,MARGALEF,ESP
9,22437,Stefano Ghisolfi,ITA,LA RAMBLA,9a+,Redpoint,SIURANA,ESP


### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

In [35]:
cql = """
SELECT *
FROM ascensos_por_encadenamiento_dificultad
WHERE tipo_encadenamiento = 'Onsight'
LIMIT 10
"""

df = execute_query(cql)
columnas = ['id_escalador',
            'nombre_escalador',
            'pais_escalador',
            'nombre_via',
            'grado_frances',
            'tipo_encadenamiento',
            'risco',
            'pais_risco']
df[columnas]

,id_escalador,nombre_escalador,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
0,22437,Stefano Ghisolfi,ITA,FISH EYE,8c,Onsight,OLIANA,ESP
1,18008,Piotr Schab,POL,PEQUENA ESTRELLA,8b+,Onsight,RODELLAR,ESP
2,6726,Manu Lopez,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
3,38626,luis rodriguez martin,ESP,BRUJO,8b,Onsight,SADERNES,ESP
4,1476,Adam Ondra,CZE,MATA HARI,8a+,Onsight,FRANKENJURA,DEU
5,4424,Marcin Wszolek,POL,NUEVE ZETA,8a+,Onsight,CHULILLA,ESP
6,16672,Gonzalo Larrocha,ESP,DNA EXTENSION,8a+,Onsight,KALYMNOS,GRC
7,27822,Michaela Kiersch,USA,LA FEMME BLANCHE,8a+,Onsight,CéüSE,FRA
8,34114,jose luis palao,ESP,REALIDAD VIRTUAL,8a+,Onsight,POLORIA,ESP
9,34114,jose luis palao,ESP,TEAM BTR,8a+,Onsight,BIELSA,ESP


### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

La obtencion de esta información mediante accesos a cassandra requiere varios accesos. Esto se debe a que se quiere visualizar 
información de varias tablas. nombre del escalador, pais, numero de ascensos en españa, grado medio y grado maximo.

- Query a la tabla de acumulados de ascensos de escaladores no españoles en españa la obtencion de los escaladores con mas ascensos en España

- Query a la tabla de dificultades y mapear la dificultad media a su grado correspondiente

- Por cada uno de los escaladores:
    - Query a la vista materializada de ascensos por escaldor y pais del risco para la obtencion de la dificultad maxima
    
    
Aprovechando al maximo la funcionalidad de la bbdd supone entonces 12 queries!

In [36]:
cql = """
SELECT *
FROM dificultades
"""
# query para la obtencion del grado medio a partir de la dificultad media
df_dificultades = execute_query(cql)

# configuro dificultad como indice del data frame para el futuro mapping
df_dificultades = df_dificultades.sort_values('dificultad',ascending=True);
df_dificultades = df_dificultades.set_index('dificultad');

In [37]:
cql = """
SELECT *
FROM ascensos_acum_noesp_esp
"""
df_ids = execute_query(cql)
df_ids = df_ids.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, True]).head(10)


cql_dif_mediaYmax = """
SELECT count(dificultad) AS num_ascensos,
       avg(dificultad) AS dificultad_media,
       max(dificultad) AS dificultad_maxima
FROM ascensos_por_escalador_paisRisco
WHERE id_escalador=%s AND pais_risco='ESP'
LIMIT 1
"""


df = pd.DataFrame()
for index,row in df_ids.iterrows():
    
    #query para obtención de grado maximo del escalador
    df_query = execute_query(cql_dif_mediaYmax, [row['id_escalador']])   
    
    # añade columnas que faltan
    df_query['nombre_escalador'] = row['nombre_escalador'];
    df_query['pais_escalador'] = row['pais_escalador'];
    
    #añade al dataframe resultado
    df = df.append(df_query, ignore_index=True)
    
df['grado_medio']=df['dificultad_media'].map(df_dificultades['grado_frances']);
df['grado_maximo']=df['dificultad_maxima'].map(df_dificultades['grado_frances']);
columnas = ["nombre_escalador", "pais_escalador", "num_ascensos", "grado_medio", "grado_maximo"]
df[columnas] 

,nombre_escalador,pais_escalador,num_ascensos,grado_medio,grado_maximo
0,Nuno Henriques,PRT,14,6a+,7a
1,"Grzegorz ""Buła"" Golowczyk",POL,9,7b/+,8b+
2,philipp kieffer,DEU,7,6c+/7a,7c
3,Wojtek Pełka,POL,7,7c+/8a,8b
4,"Gonçalo ""Gongas"" Coutinho",PRT,6,5,6a
5,Tieme van Veen,NLD,6,6c+/7a,8a
6,Kuba Pe,POL,6,7a,7b+
7,Gabriel Korbiel,POL,6,7a+/7b,7c
8,Amber Thornton,GBR,6,5c,6c
9,Benjamin Thomas,FRA,6,6c+/7a,7c+


### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

In [38]:
cql = """
SELECT *
FROM ascensos_acum_esp_esp
"""
df_ids = execute_query(cql)
df_ids = df_ids.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, True]).head(10)


cql_dif_mediaYmax = """
SELECT count(dificultad) AS num_ascensos,
       avg(dificultad) AS dificultad_media,
       max(dificultad) AS dificultad_maxima
FROM ascensos_por_escalador_paisRisco
WHERE id_escalador=%s AND pais_risco='ESP'
LIMIT 1
"""


df = pd.DataFrame()
for index,row in df_ids.iterrows():
    
    #query para obtención de grado maximo del escalador
    df_query = execute_query(cql_dif_mediaYmax, [row['id_escalador']])   
    
    # añade columnas que faltan
    df_query['nombre_escalador'] = row['nombre_escalador'];
    df_query['pais_escalador'] = row['pais_escalador'];
    
    #añade al dataframe resultado
    df = df.append(df_query, ignore_index=True)
    
df['grado_medio']=df['dificultad_media'].map(df_dificultades['grado_frances']);
df['grado_maximo']=df['dificultad_maxima'].map(df_dificultades['grado_frances']);
columnas = ["nombre_escalador", "pais_escalador", "num_ascensos", "grado_medio", "grado_maximo"]
df[columnas] 

,nombre_escalador,pais_escalador,num_ascensos,grado_medio,grado_maximo
0,Raúl Crespo,ESP,18,6a+/6b,7b
1,Chaken Gómez conde,ESP,16,7c/+,8b
2,Adrian Alameda,ESP,14,6c/+,8a+
3,Alex Garriga,ESP,13,7b+/7c,8a+
4,Xavier Gatell Romero,ESP,12,7b+,8a
5,jose luis palao,ESP,12,8a+,8b+
6,Jose Agustí,ESP,10,7c/+,8a
7,Gonzalo Larrocha,ESP,10,8a+/8b,9a
8,Tomata Tomata,ESP,10,6c/+,7a+
9,luis rodriguez martin,ESP,10,8a/+,8b+


### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

In [39]:
cql = """
SELECT count(dificultad) AS num_ascensos, avg(dificultad) AS dif_media, max(dificultad) AS dif_max
FROM ascensos_por_encadenamiento_aniocomienzo
WHERE tipo_encadenamiento IN ('Redpoint', 'Onsight','Flash')
      AND anio_comienzo > 2014
      AND anio_comienzo <= 2017
"""
df = execute_query(cql)

df['grado_medio']=df['dif_media'].map(df_dificultades['grado_frances']);
df['grado_maximo']=df['dif_max'].map(df_dificultades['grado_frances']);

df[['num_ascensos', 'grado_medio', 'grado_maximo']]


,num_ascensos,grado_medio,grado_maximo
0,590,6a/+,8a


### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

In [40]:
cql = """
SELECT count(dificultad) AS num_ascensos, avg(dificultad) AS dif_media, max(dificultad) AS dif_max
FROM ascensos_por_encadenamiento_aniocomienzo
WHERE tipo_encadenamiento IN ('Redpoint', 'Onsight','Flash')
      AND anio_comienzo > 1987
      AND anio_comienzo <= 2007
"""
df = execute_query(cql)

df['grado_medio']=df['dif_media'].map(df_dificultades['grado_frances']);
df['grado_maximo']=df['dif_max'].map(df_dificultades['grado_frances']);

df[['num_ascensos', 'grado_medio', 'grado_maximo']]


,num_ascensos,grado_medio,grado_maximo
0,3818,7a,9c


### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

In [41]:
cql = """
SELECT *
FROM ascensos_acum_pais_risco
WHERE pais_risco='ESP'
"""
df = execute_query(cql)

# las tablas de acumulacion no se pueden ordenar por el counter, por lo que tengo que hacerlo con python
df.sort_values(['num_ascensos', 'risco'], ascending=[False,True]).head(10)



,pais_risco,risco,dificultad_acum,num_ascensos
171,ESP,MARGALEF,9947,194
88,ESP,CUENCA,7350,148
281,ESP,SIURANA,7379,147
3,ESP,ALBARRACíN,6302,137
251,ESP,RODELLAR,7390,136
73,ESP,CHULILLA,6738,134
166,ESP,MALLORCA,2865,63
152,ESP,LA PEDRIZA,2623,61
189,ESP,MONTSERRAT,2811,58
98,ESP,EL CHORRO,2059,44


### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

In [42]:
cql = """
SELECT *
FROM ascensos_acum_pais_risco
WHERE pais_risco='ESP'
"""
df = execute_query(cql)

df["dificultad_media"] = (df["dificultad_acum"]/df["num_ascensos"]).round().astype(int)

#mapping de dificultad media a grado medio
df['grado_medio']=df['dificultad_media'].map(df_dificultades['grado_frances']);

columnas = ['risco', 'dificultad_media', 'grado_medio', 'num_ascensos']
df.sort_values(['dificultad_media', 'num_ascensos', 'risco'], ascending=[False, False, True]).head(10)[columnas]


,risco,dificultad_media,grado_medio,num_ascensos
142,LA COVA DE L'OCELL,70,8c,2
224,POLORIA,66,8b,2
110,FIGOLS,66,8b,1
201,ORIHUELA,65,8a+/8b,2
272,SANTA LINYA,64,8a+,18
196,OLIANA,64,8a+,14
112,FORADA,64,8a+,3
149,LA MUELA,64,8a+,2
26,BALTZOLA,64,8a+,1
44,CACIN,64,8a+,1


### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente

In [43]:
cql = """
SELECT *
FROM ascensos_acum_pais_risco
WHERE pais_risco='ESP'
"""
df = execute_query(cql)

df["dificultad_media"] = (df["dificultad_acum"]/df["num_ascensos"]).round().astype(int)

#mapping de dificultad media a grado medio
df['grado_medio']=df['dificultad_media'].map(df_dificultades['grado_frances']);

columnas = ['risco', 'dificultad_media', 'grado_medio', 'num_ascensos']
df.sort_values(['dificultad_media', 'num_ascensos', 'risco'], ascending=[True, False, True]).head(10)[columnas]

,risco,dificultad_media,grado_medio,num_ascensos
228,PORTOMARIN,21,4a,2
182,MONT-RAL,25,4c,1
203,OTO,25,4c,1
232,PUERTO ROQUE,26,4c+,3
230,PUEBLA DE GUZMAN,27,4+,5
25,BALNEARIO DE PANTICOSA,27,4+,2
277,SAVASSONA,28,5,2
143,LA DEHESA DE LA OLIVA,29,5a,1
153,LA PUENTE,29,5a,1
167,MALLORKA,29,5a,1
